# Concrete slab base - Check shear capacity of the slab 

In [189]:
#import sympy as sp
#import matplotlib.pyplot as plt
import math

Check of punching shear capacity of a slab with one layer of reinforcement. Based on 6.2.2 of EN1992-1-1:2004 for concrete members.

In [190]:
import ipywidgets as widgets
from IPython.display import display

# Global dictionary to store widget values for persistence
if 'stored_values' not in globals():
    stored_values = {}

# Dictionary to store default values, units, and descriptions
form_data = {
    'NSd_uplift': {'value': 15.0, 'unit': '[kN]', 'description': 'factored uplift force'},
    'NSd_downwards': {'value': 10.0, 'unit': '[kN]', 'description': 'factored downwards force'},
    'fck': {'value': 25, 'unit': '[MPa]', 'description': 'concrete grade'},
    'slab_depth': {'value': 150, 'unit': '[mm]', 'description': 'overall slab depth'},
    'Area_reinforcement': {'value': 252, 'unit': '[mm^2]', 'description': 'area of reinforcement'},
    'Dist_reinforcement_from_btm': {'value': 50, 'unit': '[mm]', 'description': 'distance of reinforcement from bottom'}
}

# Create widgets for each field
widgets_dict = {}
form_rows = []
for key, data in form_data.items():
    # Use stored value if available, otherwise use default
    initial_value = stored_values.get(key, data['value'])
    
    # Create FloatText widget for numeric input
    input_widget = widgets.FloatText(
        value=initial_value,
        layout={'width': '100px'}  # Fixed width for input field
    )
    widgets_dict[key] = input_widget
    
    # Adjust label width based on label length (approximate 10px per character)
    label_width = f"{len(key) * 10 + 20}px"
    label_widget = widgets.Label(
        value=key,
        layout={'width': label_width}
    )
    
    # Unit label
    unit_widget = widgets.Label(
        value=data['unit'],
        layout={'width': '50px'}
    )
    
    # Description label
    desc_widget = widgets.Label(
        value=data['description'],
        layout={'width': 'auto'}
    )
    
    # Create HBox for the row: label, input, unit, description
    row = widgets.HBox([
        label_widget,
        input_widget,
        unit_widget,
        desc_widget
    ])
    form_rows.append(row)
    
    # Define handler to update global variable and stored_values on change
    def make_handler(k):
        def update_global(change):
            globals()[k] = change['new']
            stored_values[k] = change['new']  # Store the new value
        return update_global
    input_widget.observe(make_handler(key), names='value')

# Organize widgets into sections
forces_box = widgets.VBox([
    widgets.HTML("<h3>Forces (factored)</h3>"),
    form_rows[0],  # NSd_uplift
    form_rows[1]   # NSd_downwards
])

slab_box = widgets.VBox([
    widgets.HTML("<h3>Slab Data</h3>"),
    form_rows[2],  # fck
    form_rows[3],  # slab_depth
    form_rows[4],  # Area_reinforcement
    form_rows[5]   # Dist_reinforcement_from_btm
])

# Display the form
display(widgets.VBox([forces_box, slab_box]))

# Assign initial values to global variables and print
print("Current values (assigned to global variables):")
for key, widget in widgets_dict.items():
    globals()[key] = widget.value
    stored_values[key] = widget.value  # Ensure initial values are stored
    print(f"{key}: {widget.value} {form_data[key]['unit']} ({form_data[key]['description']})")

Current values (assigned to global variables):
NSd_uplift: 25.0 [kN] (factored uplift force)
NSd_downwards: 10.0 [kN] (factored downwards force)
fck: 35.0 [MPa] (concrete grade)
slab_depth: 120.0 [mm] (overall slab depth)
Area_reinforcement: 254.0 [mm^2] (area of reinforcement)
Dist_reinforcement_from_btm: 50.0 [mm] (distance of reinforcement from bottom)


#### Shear Check

In [191]:
# d...is the mean of the effective depths in the orthogonal directions, [mm]
d = slab_depth - Dist_reinforcement_from_btm

# bw...the smallest width of the cross-section in the tensile area, [mm]
# distance assumed conservatively (given by distance of anchor from the edge of slab)
bw = 300  

gammaC = 1.5
fcd = fck / gammaC
k = min(2, 1 + (200 / (d)**(0.5)))
nuMin = 0.035 * (k)**(1.5) * (fck)**(0.5)
Asl = Area_reinforcement
rho_l = min(Asl / bw / d, 0.02)

NEd = 0   ##axial force in the cross-section due to loading or prestressing
Ac = 1    ##cross-sectional area of concrete, not assigned as NEd is zero anyway
sigma_cp = min(NEd / Ac, 0.2 * fcd)
if NEd == 0:
    sigma_cp = 0.2 * fcd
k1 = 0.15
CRdc = 0.18 / gammaC
VRdc = (min((nuMin + k1 * sigma_cp) * bw * d, (CRdc * k * (100 * rho_l * fck)**(0.3333) + k1 * sigma_cp) * bw * d)) / 1000

In [192]:
print("bw                       = ", ("%2.2f" % (bw)), "[mm]")
print("gammaC                   = ", ("%2.2f" % (gammaC)), "[-]")
print("fcd                      = ", ("%2.2f" % (fcd)), "[-]")
print("k = min(2, 1 + (200 / (d)**(0.5)))")
print("k                        = ", ("%2.2f" % (k)), "[-]")
print("rho_l = min(Asl / bw / d, 0.02)")
print("rho_l                    = ", ("%2.2f" % (rho_l)), "[-]")
print("sigma_cp                 = ", ("%2.2f" % (sigma_cp)), "[-]")
print("CRdc                     = ", ("%2.2f" % (CRdc)), "[-]")
print("VRdc                     = ", ("%2.2f" % (VRdc)), "[kN]")

bw                       =  300.00 [mm]
gammaC                   =  1.50 [-]
fcd                      =  23.33 [-]
k = min(2, 1 + (200 / (d)**(0.5)))
k                        =  2.00 [-]
rho_l = min(Asl / bw / d, 0.02)
rho_l                    =  0.01 [-]
sigma_cp                 =  4.67 [-]
CRdc                     =  0.12 [-]
VRdc                     =  27.00 [kN]


In [193]:
shear = max(NSd_downwards,NSd_uplift)
UF = shear/VRdc
print("UF                       = ", ("%2.2f" % (UF)), "[-]")
if VRdc > NSd_uplift:
    print("PASS")
else:
    print("FAIL")

UF                       =  0.93 [-]
PASS
